### Sleep state detection

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from neuropy import plotting
import subjects

sessions = subjects.nsd.ratVday1

In [ ]:
from neuropy.analyses import detect_brainstates_epochs

for sub, sess in enumerate(sessions):
    signal = sess.eegfile.get_signal()
    brainstates = detect_brainstates_epochs(
        signal,
        probe=sess.probegroup,
        theta_channel=80,
        delta_channel=82,
        sigma=1,
        ignore_epochs=sess.artifact,
    )
    brainstates.save(sess.filePrefix.with_suffix('.brainstates'))


### View brainstates detection in ephyviewer

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from neuropy import plotting
from brainstates_editor import editor
from ephyviewer import mkQApp
import subjects

app = mkQApp()
sess = subjects.nsd.ratVday1[0]
sigs = sess.eegfile.get_signal(20)

win = editor(
    sess.brainstates,
    sigs,
    filename=sess.filePrefix.with_suffix(".test"),
    paradigm=sess.paradigm,
)
win.show()
app.exec_()


PyQt5
<class 'PyQt5.QtCore.Qt'>
None
<module 'PyQt5.QtCore' from '/home/bapung/miniconda3/envs/data_analysis/lib/python3.9/site-packages/PyQt5/QtCore.so'>
None
<module 'PyQt5.QtGui' from '/home/bapung/miniconda3/envs/data_analysis/lib/python3.9/site-packages/PyQt5/QtGui.so'>
None
<module 'PyQt5.QtWidgets' from '/home/bapung/miniconda3/envs/data_analysis/lib/python3.9/site-packages/PyQt5/QtWidgets.so'>
<class 'PyQt5.QtWidgets.QWidget'>
<class 'PyQt5.QtCore.Qt'>
None
<module 'PyQt5.QtCore' from '/home/bapung/miniconda3/envs/data_analysis/lib/python3.9/site-packages/PyQt5/QtCore.so'>
<class 'PyQt5.QtCore.pyqtSignal'>
<class 'PyQt5.QtCore.Qt'>
None
<module 'PyQt5.QtCore' from '/home/bapung/miniconda3/envs/data_analysis/lib/python3.9/site-packages/PyQt5/QtCore.so'>
None
<module 'PyQt5.QtGui' from '/home/bapung/miniconda3/envs/data_analysis/lib/python3.9/site-packages/PyQt5/QtGui.so'>
None
<module 'PyQt5.QtWidgets' from '/home/bapung/miniconda3/envs/data_analysis/lib/python3.9/site-packages/

/home/bapung/Documents/Codes/pythonprogs/DataPaths/spectrogramviewer_custom.py:222: RuntimeWarning: divide by zero encountered in log10
  Sxx = 10.0 * np.log10(Sxx)


 1250
refresh duration for traces 0.00011920928955078125 s
refresh duration for brainstates 0.0004296302795410156 s
refresh duration for wvlt 0.00034546852111816406 s
refresh duration for spectrogram 0.00034165382385253906 s
2500 1250
2500 1250
refresh duration for traces 0.00011110305786132812 s
refresh duration for brainstates 0.00039315223693847656 s
refresh duration for wvlt 0.0004076957702636719 s
refresh duration for spectrogram 0.00033855438232421875 s
refresh duration for paradigm 0.0002503395080566406 s
2500 1250
<class 'PyQt5.QtCore.Qt'>
None
<module 'PyQt5.QtCore' from '/home/bapung/miniconda3/envs/data_analysis/lib/python3.9/site-packages/PyQt5/QtCore.so'>
None
<module 'PyQt5.QtGui' from '/home/bapung/miniconda3/envs/data_analysis/lib/python3.9/site-packages/PyQt5/QtGui.so'>
<class 'PyQt5.QtGui.QColor'>
<class 'PyQt5.QtCore.Qt'>
None
<module 'PyQt5.QtCore' from '/home/bapung/miniconda3/envs/data_analysis/lib/python3.9/site-packages/PyQt5/QtCore.so'>
<class 'PyQt5.QtCore.QRe

0

### Sleep fraction during 5 hours following MAZE

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from neuropy import plotting
import subjects

sessions = subjects.sd.brainstates_sess + subjects.nsd.brainstates_sess

In [ ]:
sleep_df = pd.DataFrame()
for sub, sess in enumerate(sessions):
    post= sess.paradigm["post"].flatten()
    period = [post[0],post[0] + 5 * 3600]
    prop = sess.brainstates.get_proportion_by_label(period[0], period[1])
    if "nan" in prop.keys():
        del prop["nan"]

    sleep_df = sleep_df.append(
        pd.DataFrame(
            {
                "prop": list(prop.values()),
                "state": list(prop.keys()),
                "grp":sess.tag,
            }
        ),
        ignore_index=True,
    )

subjects.GroupData().save(sleep_df,'brainstates_proportion')

### Combining brainstates into dataframe for plotting in sd_figure3
- only POST epoch and in Zt time

In [9]:
import numpy as np
from subjects import sd,nsd,GroupData
import pandas as pd

sessions = (
    sd.ratKday1
    + sd.ratNday1
    + sd.ratSday3
    + sd.ratUday4
    + sd.ratVday2
    + sd.ratRday2

    + nsd.ratKday2
    + nsd.ratNday2
    + nsd.ratSday2
    + nsd.ratUday2
    + nsd.ratVday1
)

In [13]:
brainstates_all = []
for sub, sess in enumerate(sessions):
    post = sess.paradigm["post"].flatten()
    states = sess.brainstates.time_slice(post[0], post[1])
    starts = (states.starts - post[0]) / 3600
    stops = (states.stops - post[0]) / 3600
    df = pd.DataFrame(
        dict(
            start=starts,
            stop=stops,
            label=states.labels,
            sub=sub,
            name=sess.animal.name,
            grp=sess.tag,
        )
    )
    brainstates_all.append(df)

brainstates_all = pd.concat(brainstates_all, ignore_index=True)

GroupData().save(brainstates_all, "ev_brainstates")


ev_brainstates saved
